In [329]:
import pandas as pd
import functools as ft
import numpy as np
from ast import literal_eval

In [330]:
FILEPATH = '../data/movie-set-data/'

In [331]:
credits_df = pd.read_csv(f'{FILEPATH}credits.csv')
keywords_df = pd.read_csv(f'{FILEPATH}keywords.csv')
links_df = pd.read_csv(f'{FILEPATH}links.csv')
links_small_df = pd.read_csv(f'{FILEPATH}links_small.csv')
movie_meta_df = pd.read_csv(f'{FILEPATH}movies_metadata.csv')
ratings_df = pd.read_csv(f'{FILEPATH}ratings.csv')
ratings_small_df = pd.read_csv(f'{FILEPATH}ratings_small.csv')

/Users/ivan/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [332]:
movie_meta_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [333]:
# dropping corrupted rows
filtered_rows = movie_meta_df[movie_meta_df['vote_count'].isnull()].index
movie_meta_df=movie_meta_df.drop(index=filtered_rows)

movie_meta_df['id'] = movie_meta_df.id.astype('int')

In [334]:
# dropping adult rated movies
filtered_rows = movie_meta_df[movie_meta_df['adult']=='True'].index
movie_meta_df = movie_meta_df.drop(index=filtered_rows)

# dropping adult column no longer needed
movie_meta_df = movie_meta_df.drop(columns='adult')

In [335]:
# dropping movies that are not already Released
filtered_rows = movie_meta_df[movie_meta_df['status']!='Released'].index
movie_meta_df = movie_meta_df.drop(index=filtered_rows)

# dropping status column no longer needed
movie_meta_df = movie_meta_df.drop(columns='status')

In [336]:
movie_meta_df['video'].unique()

array([False, True], dtype=object)

In [337]:
# dropping movies that are not already Released
filtered_rows = movie_meta_df[movie_meta_df['video']==True].index
movie_meta_df = movie_meta_df.drop(index=filtered_rows)

# dropping status column no longer needed
movie_meta_df = movie_meta_df.drop(columns='video')

In [338]:
# removing appended imdb_id
movie_meta_df['imdb_id'] = movie_meta_df['imdb_id'].astype(str).str.replace('tt0', '')
movie_meta_df['imdb_id'] = movie_meta_df['imdb_id'].astype(str).str.replace('tt', '')
movie_meta_df.rename(columns={'imdb_id':'imdbId'}, inplace=True)

# merging meta and links df on imdb value
filtered_rows = movie_meta_df[movie_meta_df['imdbId'] == 'nan'].index
movie_meta_df = movie_meta_df.drop(index=filtered_rows)

movie_meta_df['imdbId'] = movie_meta_df['imdbId'].astype('int64')
movie_meta_df = pd.merge(movie_meta_df,links_df,on='imdbId')

# Cleaning JSON columns

In [339]:
# merging datasets for text mining
dataframes = [keywords_df,credits_df,movie_meta_df]
text_df = ft.reduce(lambda left, right: pd.merge(left, right, on='id'), dataframes)

In [340]:
movies_df = pd.DataFrame(text_df[['movieId', 'keywords','cast', 'crew', 'genres',
                     'popularity', 'production_companies',
                     'production_countries','title', 'revenue']])

# dropping duplicates and missing values
movies_df.drop_duplicates(keep=False,inplace=True)
movies_df.dropna(inplace=True)

In [341]:
features = ['cast', 'crew', 'genres', 'keywords', 'production_companies', 'production_countries']

movies_df = movies_df.dropna(subset=features)

for feature in features:
    movies_df[feature] = movies_df[feature].apply(literal_eval)

In [342]:
## function to get name of director from the crew field
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

# get director
movies_df['director'] = movies_df['crew'].apply(get_director)
movies_df = movies_df.drop(columns='crew')

In [343]:
# Returns the list of top 3 elements for genres and keywords
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [344]:
## function to get name of production company from the crew field
def get_names(x):
    for i in x:
        return i['name']
    return np.nan

# get production company names
movies_df['prod_company'] = movies_df['production_companies'].apply(get_names)
movies_df['prod_country'] = movies_df['production_countries'].apply(get_names)

movies_df = movies_df.drop(columns='production_companies')
movies_df = movies_df.drop(columns='production_countries')

In [345]:
features = ['cast', 'keywords', 'genres']

for feature in features:
    movies_df[feature] = movies_df[feature].apply(get_list)

In [346]:
filtered_rows = movies_df[movies_df['prod_country'] != 'United States of America'].index
movies_df = movies_df.drop(index=filtered_rows)
movies_df = movies_df.drop(columns='prod_country')

In [347]:
movies_df.head()

,movieId,keywords,cast,genres,popularity,title,revenue,director,prod_company
0,1,"[jealousy, toy, boy]","[Tom Hanks, Tim Allen, Don Rickles]","[Animation, Comedy, Family]",21.946943,Toy Story,373554033.0,John Lasseter,Pixar Animation Studios
1,2,"[board game, disappearance, based on children'...","[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[Adventure, Fantasy, Family]",17.015539,Jumanji,262797249.0,Joe Johnston,TriStar Pictures
2,3,"[fishing, best friend, duringcreditsstinger]","[Walter Matthau, Jack Lemmon, Ann-Margret]","[Romance, Comedy]",11.7129,Grumpier Old Men,0.0,Howard Deutch,Warner Bros.
3,4,"[based on novel, interracial relationship, sin...","[Whitney Houston, Angela Bassett, Loretta Devine]","[Comedy, Drama, Romance]",3.859495,Waiting to Exhale,81452156.0,Forest Whitaker,Twentieth Century Fox Film Corporation
4,5,"[baby, midlife crisis, confidence]","[Steve Martin, Diane Keaton, Martin Short]",[Comedy],8.387519,Father of the Bride Part II,76578911.0,Charles Shyer,Sandollar Productions


# Use avg rating to fill for movies & prep modeling dataframe

In [348]:
# averaging ratings and merging ratings to final dataframe on movie Id
ratings_df.drop(columns=['userId', 'timestamp'], inplace=True)
avg_ratings = round(ratings_df.groupby('movieId').mean(),2)
final_df = pd.merge(movies_df,avg_ratings,on='movieId')
final_df.rename(columns={'rating':'avg_rating'}, inplace=True)

# dropping null values
final_df = final_df.dropna()

# popularity clean up
final_df['popularity'] = final_df['popularity'].astype('float')
final_df['popularity'] = round(final_df['popularity'],2)

# reducing revenue to millions to mitigate large values
final_df['revenue_millions'] = round(final_df['revenue']/1000000, 3)
final_df = final_df.drop(columns='revenue')

# Exporting dataframe for modeling

In [351]:
final_df.to_csv('../data/iac_dataset/modeling_data.csv',index=False)